In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,TensorDataset
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.is_available())

cpu
False


In [16]:
def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)


    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))
   

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

   

    return train_data, train_label, test_data, test_label

In [17]:
train_data, train_label, test_data, test_label = read_bci_data()
dataset = TensorDataset(torch.from_numpy(train_data),torch.from_numpy(test_data))
loader_train = DataLoader(dataset,batch_size=256,shuffle=True,num_workers=4)
print(loader_train)
dataset = TensorDataset(torch.from_numpy(test_data),torch.from_numpy(test_label))
loader_test = DataLoader(dataset,batch_size=256,shuffle=False,num_workers=4)

In [25]:
class EEGNet(nn.Module):
    def __init__(self,activation=nn.ELU()):
        super(EEGNet,self).__init__()
        self.firstconv=nn.Sequential(
            nn.Conv2d(1,16,kernel_size=(1,51),stride=(1,1),padding=(0,25),bias=False),
            nn.BatchNorm2d(16,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True)
        )
        self.depthwiseConv=nn.Sequential(
            nn.Conv2d(16,32,kernel_size=(2,1),stride=(1,1),groups=16,bias=False),
            nn.BatchNorm2d(32,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True),
            activation,
            nn.AvgPool2d(kernel_size=(1,4),stride=(1,4),padding=0),
            nn.Dropout(p=0.25)
        )
        self.seperableConv=nn.Sequential(
            nn.Conv2d(32,32,kernel_size=(1,15),stride=(1,1),padding=(0,7),bias=False),
            nn.BatchNorm2d(32,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True),
            activation,
            nn.AvgPool2d(kernel_size=(1,8),stride=(1,8),padding=0),
            nn.Dropout(p=0.25)
        )
        self.classify=nn.Linear(736,2)
    def forward(self,inputs):
        output = self.firstconv(inputs)
        output = self.depthwiseConv(output)
        output = self.seperableConv(output)
        return self.classify(output)
    def calculate_accuracy(self, pred_y, label):
        pred_y = pred_y.reshape(-1,1)
        pred_y = np.round(pred_y) 
        correct = np.sum(pred_y == label)
        total = label.shape[0]  
        accuracy = (correct / total) * 100  
        return accuracy
    
    
    def train_and_test(self, loader_train,loader_test):
        batch_size= 64        
        lr = 1e-2        
        activations={nn.ReLU(),nn.LeakyReLU(),nn.ELU()}
        loss = torch.nn.CrossEntropyLoss()
        train_acc = []
        
        for act in activations:
            model=EEGNet(act)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            for epoch in range(150):
                model.train()
                for idx,(data,label) in enumerate(loader_train):
                    print(data)
                    data=data.to(device,dtype=torch.float)
                    label = label.to(device,dtype = torch.long)
                    pred_y = model(data)
                    loss = loss(label, pred_y)
                    accuracy = self.calculate_accuracy(pred_y, train_label)
                    if epoch % 100 == 0:
                        print(f"Epoch: {epoch}, Loss: {np.mean(loss)}, Accuracy: {accuracy:.2f}%")
            print("Training finished.")
        print('start testing:')
        model.eval()
        for index, data in enumerate(test_data):
            predict = model(data)
            loss = loss(test_label, predict)
            accuracy = self.calculate_accuracy(pred_y, test_label)
        print(f"Loss: {np.mean(loss)}, Accuracy: {accuracy:.2f}%")
        print('testing finished')
    
    
neural_network = EEGNet()
neural_network.train_and_test(loader_train,loader_test)

tensor([[[[-7.7302, -8.4823, -7.4358,  ...,  8.6822,  8.6364,  5.4067],
          [-3.4430, -5.0863, -2.3353,  ...,  6.6365,  6.8867, -0.6035]]],


        [[[ 0.1434,  2.1984,  2.6896,  ..., -8.2459, -8.4762, -4.0367],
          [ 2.7808,  4.1204,  3.2492,  ..., -1.1802,  0.2678,  4.4027]]],


        [[[-5.4738, -5.4845, -5.9239,  ...,  1.8124,  1.7133, -1.0405],
          [ 5.1885, -2.6938, -9.5828,  ..., -5.6463, -5.0207, -5.1351]]],


        ...,


        [[[ 9.3532,  9.2338,  7.3907,  ..., -4.9905, -5.0284, -3.2970],
          [ 4.5744,  3.9874,  1.1172,  ...,  3.9465,  0.7253, -3.2857]]],


        [[[ 2.7232,  1.0633,  1.4646,  ..., -5.6768, -3.2160,  2.6347],
          [ 2.5397, -0.7774, -0.6935,  ...,  2.3993,  3.9724,  5.4906]]],


        [[[ 7.0467,  2.7568, -3.0466,  ...,  8.3847, 11.0118,  9.5945],
          [ 6.4443,  2.2773, -1.2458,  ...,  7.6924,  6.0704,  4.4515]]]],
       dtype=torch.float64)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8192x23 and 736x2)

In [19]:
neural_network = EEGNet()
neural_network.train_and_test(train_data,train_label, test_data, test_label)

ReLU()


TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [32;1mint[0m)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [32;1mint[0m)


In [15]:
activations={nn.ReLU(),nn.LeakyReLU(),nn.ELU()}
for activation in activations:
    print (activation)

ReLU()
LeakyReLU(negative_slope=0.01)
ELU(alpha=1.0)
